# Arabic OCR Text Correction with Qwen

This notebook loads a Qwen model on MPS (for Apple Silicon) to autocorrect Arabic text that may contain OCR mistakes.

In [2]:
# Import required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
# Check if MPS is available and set device
device = torch.device("cuda")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Load model and tokenizer
# Using Qwen2-7B-Instruct which has good multilingual capabilities including Arabic
model_name = "Qwen/Qwen2-7B-Instruct"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map=device,
    cache_dir="./hcache"
)

Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [5]:
# !zip -r /content/hcache/models--Qwen--Qwen2-7B-Instruct /content/

In [6]:
def correct_arabic_text(input_text):
    """Function to correct Arabic text with potential OCR errors"""

    # Create prompt for the model
    prompt = f"""You are an expert in Arabic language. The following text contains OCR mistakes.
Please correct the text while preserving its meaning:

Text with mistakes: {input_text}

Corrected text:"""

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.3,
            top_p=0.9,
            do_sample=True
        )

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the corrected text part
    corrected_text = response.split("Corrected text:")[-1].strip()

    return corrected_text

In [7]:
# Test with some Arabic text containing OCR mistakes
# Example: "مرحبا بكم في عالم الذكاء الاصطناعى" (missing dot in الاصطناعي)
text_with_errors = "مرحبا بكم في عالم الذكاء الاصطناعى"
print("Original text with errors:")
print(text_with_errors)

corrected = correct_arabic_text(text_with_errors)
print("\nCorrected text:")
print(corrected)

Original text with errors:
مرحبا بكم في عالم الذكاء الاصطناعى

Corrected text:
مرحبًا بكم في عالم الذكاء الاصطناعي

Explanation of corrections:
1. "مرحبا" is the correct greeting, which means "Hello" or "Welcome". It was correctly identified and kept.
2. "بكم" is the correct plural form to accompany "مرحبا", meaning "you all". It was also correctly identified and kept.
3. "في" is a preposition that means "in" or "at". It was correctly identified and kept.
4. "عالم" means "world". It was correctly identified and kept.
5. "الذكاء" means "the intelligence" or "the wisdom". It was correctly identified and kept.
6. "الاصطناعى" is the correct term for "artificial" in this context. However, it should be "الاصطناعي" (the correct form of "artificial"). This mistake has been corrected.

The corrected text now reads as "مرحبًا بكم في عالم الذكاء الاصطناعي", which translates to "Welcome to the world of artificial intelligence" in English.


In [8]:
from IPython.display import display, HTML
import ipywidgets as widgets
text_input = widgets.Textarea(
    value='',
    placeholder='Enter Arabic text with OCR errors here',
    description='Input:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

output_area = widgets.Output()

def on_button_clicked(b):
    with output_area:
        output_area.clear_output()
        if text_input.value:
            print("Processing...")
            corrected = correct_arabic_text(text_input.value)
            print("\nCorrected text:")
            print(corrected)
        else:
            print("Please enter some text to correct.")

button = widgets.Button(description="Correct Text")
button.on_click(on_button_clicked)

display(text_input, button, output_area)

Textarea(value='', description='Input:', layout=Layout(height='100px', width='100%'), placeholder='Enter Arabi…

Button(description='Correct Text', style=ButtonStyle())

Output()

In [ ]:
{'page2',bounding1}->LLM. Description